In [ ]:
from operator import index

from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A, PART_OF
import pandas as pd
from linkml_runtime import SchemaView
from linkml_runtime.dumpers import yaml_dumper
import requests
import pprint

from typing import Dict, Any
import csv

from sklearn.feature_extraction.text import CountVectorizer  # from scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.sparse import hstack

import re

import os

import duckdb
import sqlite3

import gzip
import shutil

from urllib.parse import urlparse

In [ ]:
# Approved prefixes (case-insensitive)
approved_prefixes = ['ENVO']

In [ ]:
# make a biomes curie -> label dict?
BIOME_CURIE = 'ENVO:00000428'

In [ ]:
ENV_BROAD_SCALE_ENUM = "EnvBroadScaleSoilEnum"

In [ ]:
ncbi_query = """
SELECT content, COUNT(1) AS sample_count 
FROM attributes 
WHERE harmonized_name = 'env_broad_scale' AND package_content = 'MIMS.me.soil.6.0' 
GROUP BY content 
ORDER BY COUNT(1) DESC
"""

In [ ]:
envo_adapter_string = "sqlite:obo:envo"

In [ ]:
# goldterms_adapter_string = "sqlite:obo:envo"

In [ ]:
goldterms_semsql_url = "https://s3.amazonaws.com/bbop-sqlite/goldterms.db.gz"

# https://s3.amazonaws.com/bbop-sqlite/
# <Contents>
# <Key>goldterms.db.gz</Key>
# <LastModified>2024-11-03T17:24:56.000Z</LastModified>
# <ETag>"fe8e35b215786cb9fc347b7fadbe055f"</ETag>
# <Size>2935781</Size>
# <StorageClass>STANDARD</StorageClass>
# </Contents>


In [ ]:
# todo could this have been done with a OAK query, eliminating the need to explicitly download the file?

goldterms_envo_query = """
SELECT
	*
FROM
	statements s
WHERE
	predicate = 'mixs:env_broad'"""

In [ ]:
GOLDTERMS_SOIL = 'GOLDTERMS:4212'

In [ ]:
goldterms_soil_subclass_query = f"""
select
	subject
from
	entailed_edge ee
where
	predicate = 'rdfs:subClassOf'
	and object = '{GOLDTERMS_SOIL}'
"""

In [ ]:
previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

In [ ]:
NMDC_RUNTIME_BASE_URL = 'https://api.microbiomedata.org/nmdcschema/'
STUDY_SET_COLLECTION = 'study_set'
BIOSAMPLE_SET_COLLECTION = 'biosample_set'

In [ ]:
env_package_override_file = '../../mam-env-package-overrides.tsv'
override_column = 'mam_inferred_env_package'

In [ ]:
# ncbi_duckdb_file = '../../ncbi_biosamples.duckdb'

ncbi_duckdb_url = 'https://portal.nersc.gov/project/m3408/biosamples_duckdb/ncbi_biosamples_2024-09-23.duckdb.gz'

In [ ]:
gold_data_url = "https://gold.jgi.doe.gov/download?mode=site_excel"
BIOSAMPLES_SHEET = "Biosample"

In [ ]:
# Initialize cache dictionaries for predict_from_normalized_env_packages
ancestor_cache = {}
descendant_cache = {}

In [ ]:
# todo is filling memory with things like this a good idea? for understandability? or performance?
# todo they should be aggregated somewhere, as specified by the config.yaml
# todo or should we going straight to data frames? in which case a dlist of dicts might be preferable
def get_curie_descendants_label_dict(curie, predicates, adapter):
    curie_label_dict = {}
    for descendant in adapter.descendants(curie, predicates=predicates):
        curie_label_dict[descendant] = adapter.label(descendant)
    return curie_label_dict

In [ ]:
def curie_descendants_label_dict_to_lod(curie_label_dict):
    return [{'curie': k, 'label': v} for k, v in curie_label_dict.items()]

In [ ]:
def curie_descendants_label_lod_to_df(curie_label_lod):
    return pd.DataFrame(curie_label_lod)

In [ ]:
def get_schemaview_from_source(source):
    return SchemaView(source)

In [ ]:
# def get_schema_from_schemaview(schemaview):
#     return schemaview.schema

In [ ]:
def parse_hierarchically_underscored_strings(hierarchically_underscored_string_list):
    result = []
    for item in hierarchically_underscored_string_list:
        # Remove leading underscores for label, split on '[' to separate curie
        label, curie = item.lstrip('_').split(' [')
        # Remove the trailing ']' from curie
        curie = curie.rstrip(']')
        # Append dictionary with label and curie
        result.append({'label': label.strip(), 'curie': curie.strip()})
    return result

In [ ]:
def dedupe_underscoreless_pvs(underscoreless_pvs):
    # Dictionary to store CURIE as key and list of unique labels as values
    curie_to_labels = {}

    for item in underscoreless_pvs:
        curie = item['curie']
        label = item['label']

        # Initialize the list if curie is not yet a key
        if curie not in curie_to_labels:
            curie_to_labels[curie] = []

        # Add label if it is not already in the list for this curie
        if label not in curie_to_labels[curie]:
            curie_to_labels[curie].append(label)
    return curie_to_labels


In [ ]:
def validate_curie_label_list_dict(curie_label_dict, adapter, print_flag=False):
    problem_curies = []
    valid_curies = []
    for curie, labels in curie_label_dict.items():
        true_label = adapter.label(curie)
        if true_label not in labels:
            problem_curies.append(curie)
            if print_flag:
                print(f"Error: {curie} has true label {true_label} which doesn't appear in {labels}")
        else:
            valid_curies.append({"curie": curie, "label": true_label})
    return {"problems": problem_curies, "valids": valid_curies}

In [ ]:
# todo could pre-determine the collection sizes
# todo could report elapsed time

def get_docs_from_nmdc_collection(base_url, collection_name, max_page_size=1000, stop_after=None):
    """
    Fetch all documents from a paginated API. Defaults to fetching a large number of documents per page.
    Optionally stop after a specified number of documents.

    Parameters:
    - base_url: The base URL of the API endpoint (e.g., 'https://api.microbiomedata.org/nmdcschema/').
    - collection_name: The name of the collection to fetch (e.g., 'biosample_set').
    - max_page_size: The maximum number of items to retrieve per page (default 1000).
    - stop_after: Optional parameter to stop fetching after a certain number of documents (default None).

    Returns:
    - A list of documents fetched from the API.
    """
    documents = []
    page_token = None
    total_documents = 0
    page_count = 0

    # Construct the full URL with the collection name
    url = f"{base_url}{collection_name}"

    while True:
        page_count += 1
        # Prepare the query parameters
        params = {
            'collection_name': collection_name,
            'max_page_size': max_page_size,  # Set large max_page_size to reduce pagination
        }

        if page_token:
            params['page_token'] = page_token  # Add the page token for pagination

        # Send the request to the API
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error fetching data: {response.status_code}")
            break

        data = response.json()

        # Add the current page of documents to the list
        num_documents_on_page = len(data['resources'])
        documents.extend(data['resources'])
        total_documents += num_documents_on_page

        # Status reporting
        print(f"Fetched page {page_count} with {num_documents_on_page} documents. Total fetched: {total_documents}")

        # If stop_after is provided, stop fetching after reaching the specified number of documents
        if stop_after and total_documents >= stop_after:
            documents = documents[:stop_after]  # Trim to the required number
            print(f"Reached stop_after limit of {stop_after} documents.")
            break

        # Check if there is a next page
        page_token = data.get('next_page_token')
        if not page_token:
            print("All documents fetched.")
            break  # Exit the loop if no more pages are available

    return documents

In [ ]:

def get_name_or_rawval(env_scale: Dict[str, Any]) -> str:
    """Safely extract label from environmental scale data."""
    if env_scale:
        term = env_scale.get('term')
        if term:
            return term.get('name', term.get('has_raw_value', ''))
    return ''

In [ ]:
def tsv_to_dict_of_dicts(tsv_file, outer_key_column):
    """
    Reads a TSV file into a dictionary of dictionaries.

    :param tsv_file: Path to the TSV file.
    :param outer_key_column: The column name or index to be used as the key for the outer dictionary.
    :return: A dictionary of dictionaries, with outer keys being the values from the specified column.
    """
    with open(tsv_file, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')

        result = {}

        for row in reader:
            outer_key = row[outer_key_column]
            result[outer_key] = {key: value for key, value in row.items() if key != outer_key_column}

    return result

In [ ]:
# todo only gets authoritative labels from the passed adapter, which is presumably EnvO only
# todo would benefit from caching of labels

def biosamples_lod_context_extractor(biosamples_lod, adapter, env_pacakge_overrides=None):
    new_lod = []
    for biosample in biosamples_lod:
        insdc_identifiers = biosample.get('insdc_biosample_identifiers', [])

        env_broad_scale_label = get_name_or_rawval(biosample.get('env_broad_scale'))
        env_local_scale_label = get_name_or_rawval(biosample.get('env_local_scale'))
        env_medium_label = get_name_or_rawval(biosample.get('env_medium'))

        # Extracting optional scalar env_package.has_raw_value
        env_package_has_raw_value = biosample.get('env_package', {}).get('has_raw_value', '')

        # Extracting required multivalued part_of
        associated_studies = '|'.join(biosample.get('associated_studies', []))  # Assuming part_of is a list of strings

        row: Dict[str, str] = {
            'id': biosample['id'],
            'insdc_biosample_identifiers': '|'.join(insdc_identifiers) if insdc_identifiers else '',

            'env_broad_scale_id': biosample['env_broad_scale']['term']['id'],
            'env_broad_scale_mongo_label': env_broad_scale_label,
            'env_broad_scale_auth_label': adapter.label(biosample['env_broad_scale']['term']['id']),

            'env_local_scale_id': biosample['env_local_scale']['term']['id'],
            'env_local_scale_mongo_label': env_local_scale_label,
            'env_local_scale_auth_label': adapter.label(biosample['env_local_scale']['term']['id']),

            'env_medium_id': biosample['env_medium']['term']['id'],
            'env_medium_mongo_label': env_medium_label,
            'env_medium_auth_label': adapter.label(biosample['env_medium']['term']['id']),

            'env_package_has_raw_value': env_package_has_raw_value,
            'normalized_env_package': 'soil' if env_package_has_raw_value == 'ENVO:00001998' else env_package_has_raw_value.lower(),
            # todo abstract this though label search, or at least providing a lookup structure

            'associated_studies': associated_studies
        }

        if env_pacakge_overrides and biosample['id'] in env_pacakge_overrides:
            print(
                f"Overriding env_package for biosample {biosample['id']} from {row['normalized_env_package']} to {env_pacakge_overrides[biosample['id']]['mam_inferred_env_package']}")
            row['normalized_env_package'] = env_pacakge_overrides[biosample['id']]['mam_inferred_env_package']

        new_lod.append(row)
    return new_lod


In [ ]:
def get_hierarchy_terms(curie: str, adapter) -> dict:
    """
    Extract ancestor and descendant terms from the ontology for a given CURIE,
    using caching to improve performance and filtering by 'is_a' relationships.

    Args:
        curie (str): CURIE identifier for the ontology term.
        adapter: Ontology adapter.

    Returns:
        dict: Dictionary containing lists of ancestor and descendant terms.
    """
    if curie in ancestor_cache:
        ancestors = ancestor_cache[curie]
    else:
        try:
            ancestors = list(adapter.ancestors(curie, predicates=[IS_A]))
            ancestor_cache[curie] = [adapter.label(ancestor) for ancestor in ancestors if ancestor]
        except Exception as e:
            print(f"Error retrieving ancestors for {curie}: {e}")
            ancestor_cache[curie] = []

    if curie in descendant_cache:
        descendants = descendant_cache[curie]
    else:
        try:
            descendants = list(adapter.descendants(curie, predicates=[IS_A]))
            descendant_cache[curie] = [adapter.label(descendant) for descendant in descendants if descendant]
        except Exception as e:
            print(f"Error retrieving descendants for {curie}: {e}")
            descendant_cache[curie] = []

    return {
        'ancestors': ancestor_cache[curie],
        'descendants': descendant_cache[curie],
    }

In [ ]:
def vectorize_terms(df, column):
    """
    Vectorize the ancestor or descendant terms for a given column.

    Args:
        df (pd.DataFrame): The input dataframe.
        column (str): The column name to vectorize.

    Returns:
        sparse matrix: The vectorized term matrix.
    """
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(
        df[column].apply(lambda x: ' '.join([str(term) for term in x if term is not None]) if x is not None else '')
    )

In [ ]:
def predict_from_normalized_env_packages(df_raw, adapter):
    # Apply the function to the relevant columns

    df = df_raw.copy()
    for column in ['env_broad_scale_id', 'env_local_scale_id', 'env_medium_id']:
        df[f'{column}_ancestors'] = df[column].apply(lambda x: get_hierarchy_terms(x, adapter)['ancestors'])
        df[f'{column}_descendants'] = df[column].apply(lambda x: get_hierarchy_terms(x, adapter)['descendants'])

    # Vectorize each set of terms separately
    broad_scale_ancestors = vectorize_terms(df, 'env_broad_scale_id_ancestors')
    broad_scale_descendants = vectorize_terms(df, 'env_broad_scale_id_descendants')

    local_scale_ancestors = vectorize_terms(df, 'env_local_scale_id_ancestors')
    local_scale_descendants = vectorize_terms(df, 'env_local_scale_id_descendants')

    medium_ancestors = vectorize_terms(df, 'env_medium_id_ancestors')
    medium_descendants = vectorize_terms(df, 'env_medium_id_descendants')

    # Combine all feature matrices
    X = hstack([
        broad_scale_ancestors,
        broad_scale_descendants,
        local_scale_ancestors,
        local_scale_descendants,
        medium_ancestors,
        medium_descendants
    ])

    # Filter the DataFrame to only include non-null rows for the target column
    df_filtered = df[df['normalized_env_package'].notnull() & (df['normalized_env_package'] != "")]

    # Extract the target variable
    y = df_filtered['normalized_env_package']

    # Ensure X corresponds to the filtered rows
    X_filtered = X[df_filtered.index]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_filtered, y, test_size=0.3, random_state=42)

    # Train a Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = clf.predict(X_test)

    # Evaluate the model
    print(classification_report(y_test, y_pred))

    # # Predict the normalized_env_package for all rows
    # df['predicted_normalized_env_package'] = clf.predict(X)

    # # If you want to add confidence scores for each class
    # class_probabilities = clf.predict_proba(X)
    # 
    # # Get the class labels from the model
    # class_labels = clf.classes_
    # 
    # # Add a column for each class with the corresponding confidence score
    # for i, class_label in enumerate(class_labels):
    #     df[f'confidence_{class_label}'] = class_probabilities[:, i]
    # 
    # return df

    return clf.predict(X)

In [ ]:
def parse_curie_label(text, approved_prefixes=['ENVO']):
    # Case-insensitive pattern for matching approved prefixes followed by an ID
    pattern = r'\b(?:' + '|'.join(approved_prefixes) + r')\s*[:_]\s*(\d+)\b'
    curie_match = re.search(pattern, text, re.IGNORECASE)
    
    if curie_match:
        curie = f"{approved_prefixes[0].upper()}:{curie_match.group(1)}"  # standardize prefix to 'ENVO:ID'
        label = re.sub(pattern, "", text).strip("[]() ")
        # replace any colons in the label with a whitespace
        return pd.Series([label, curie])
    else:
        label = re.sub(r':', ' ', text)
        return pd.Series([label, None])  # No CURIE found, return original label and None for CURIE


In [ ]:
def get_longest_annotation_curie(text, adapter):
    annotations = adapter.annotate_text(text)
    if not annotations:  # Check if annotations list is empty
        return None
    try:
        longest_annotation = max(annotations, key=lambda x: x.subject_end - x.subject_start)
        return longest_annotation.object_id
    except ValueError:
        return None  # Return None if there's an unexpected issue with finding the max


Can this be reconstructed in a traceable and reusable manner quickly? 

If so, then reuse this for generating water and sediment broad scale voting sheets.

See https://github.com/microbiomedata/external-metadata-awareness/blob/main/Makefiles/soil-env_broad_scale.Makefile

and all of it's dependencies

**or this newer, un-merged version**

```makefile
local/soil-env-broad-scale-evidence-table.tsv: config/soil-env_broad_scale-evidence-config.yaml \
local/biome-ids.tsv \
local/EnvBroadScaleSoilEnum-pvs-keys-parsed-unique.csv \
local/nmdc-production-biosamples-soil-env_broad_scale.tsv \
local/ncbi-mims-soil-biosamples-env_broad_scale-annotated.tsv \
local/goldData_biosamples-inferred-soil-env_broad_scale-counts.tsv
	$(RUN) python external_metadata_awareness/extract_value_set_evidence.py \
		--config $< \
		--downsample-uncounted \
		--output-file $@
```



```config/soil-env_broad_scale-evidence-config.yaml``` from the same unmerged branch:

```yaml
- filename: local/biome-ids.tsv
  output_prefix: all_biomes_oak
  header: false
  data_column_number: 1
- filename: local/EnvBroadScaleSoilEnum-pvs-keys-parsed-unique.csv
  output_prefix: historical_permissible_values
  header: true
  data_column_name: normalized_curie
- filename: local/nmdc-production-biosamples-soil-env_broad_scale.tsv
  output_prefix: NMDC_soil
  header: false
  data_column_number: 1
  count_column_number: 2
- filename: local/ncbi-mims-soil-biosamples-env_broad_scale-annotated.tsv
  output_prefix: NCBI_mims_soil_trusting_CURIe
  header: true
  data_column_name: normalized_curie
  count_column_name: count
- filename: local/ncbi-mims-soil-biosamples-env_broad_scale-annotated.tsv
  output_prefix: NCBI_mims_soil_trusting_labels
  header: true
  data_column_name: matched_id
  count_column_name: count
- filename: local/goldData_biosamples-inferred-soil-env_broad_scale-counts.tsv
  output_prefix: GOLD_env_terr_soil
  header: false
  data_column_number: 1
  count_column_number: 2
```

In [ ]:
# Determine the filenames and target directory
ncbi_compressed_filename = urlparse(ncbi_duckdb_url).path.split('/')[-1]
ncbi_filename = os.path.splitext(ncbi_compressed_filename)[0]
target_dir = os.path.join("..", "..")  # Two levels up

In [ ]:
# Fetch the contents from the URL and save compressed file in target directory
ncbi_response = requests.get(ncbi_duckdb_url)
ncbi_compressed_file_path = os.path.join(target_dir, ncbi_compressed_filename)
with open(ncbi_compressed_file_path, "wb") as f:
    f.write(ncbi_response.content)
    
# ~ 2 minutes @ 250 Mbps

In [ ]:
# Unzip the compressed file and save the extracted file in target directory
ncbi_uncompressed_file_path = os.path.join(target_dir, ncbi_filename)
with gzip.open(ncbi_compressed_file_path, "rb") as f_in:
    with open(ncbi_uncompressed_file_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
        
# ~ 1 minute

In [ ]:
ncbi_conn = duckdb.connect(database=ncbi_uncompressed_file_path, read_only=True)

In [ ]:
envo_adapter = get_adapter(envo_adapter_string)

In [ ]:
biome_descendants = get_curie_descendants_label_dict(BIOME_CURIE, [IS_A], envo_adapter)

In [ ]:
biome_descendants_lod = curie_descendants_label_dict_to_lod(biome_descendants)

In [ ]:
biome_descendants_frame = curie_descendants_label_lod_to_df(biome_descendants_lod)

In [ ]:
biome_descendants_frame

**Use `biome_descendants_frame` as an approximation of `local/biome-ids.tsv`**

In [ ]:
sv = get_schemaview_from_source(previous_submission_schema_url)

In [ ]:
soil_env_broad_scale_enum = sv.get_enum(ENV_BROAD_SCALE_ENUM)
soil_env_broad_scale_pvs_keys = list(soil_env_broad_scale_enum.permissible_values.keys())

In [ ]:
initially_parsed_soil_env_broad_scale_pvs = parse_hierarchically_underscored_strings(soil_env_broad_scale_pvs_keys)

In [ ]:
deduped_soil_env_broad_scale_pvs = dedupe_underscoreless_pvs(initially_parsed_soil_env_broad_scale_pvs)

In [ ]:
pv_validation_results = validate_curie_label_list_dict(deduped_soil_env_broad_scale_pvs, envo_adapter, print_flag=True)

In [ ]:
pv_validation_results

**Use `pv_validation_results['valids']` as an approximation of `local/EnvBroadScaleSoilEnum-pvs-keys-parsed-unique.csv`**

In [ ]:
all_biosamples = get_docs_from_nmdc_collection(NMDC_RUNTIME_BASE_URL,
                                               BIOSAMPLE_SET_COLLECTION)  # Example with stop_after

In [ ]:
all_studies = get_docs_from_nmdc_collection(NMDC_RUNTIME_BASE_URL, STUDY_SET_COLLECTION)  # Example with stop_after

In [ ]:
env_pacakge_overrides = tsv_to_dict_of_dicts(env_package_override_file, 'id')

In [ ]:
# env_pacakge_overrides
# todo or show as frame
# todo include some other columns for context?

In [ ]:
biosample_contexts_lod = biosamples_lod_context_extractor(all_biosamples, envo_adapter,
                                                          env_pacakge_overrides=env_pacakge_overrides)

In [ ]:
biosample_contexts_frame = pd.DataFrame(biosample_contexts_lod)

In [ ]:
# print a value count for the normalized_env_package column
print("Value counts for normalized_env_package column:")
print(biosample_contexts_frame['normalized_env_package'].value_counts(dropna=False))

In [ ]:
package_predictions = predict_from_normalized_env_packages(biosample_contexts_frame, envo_adapter)

In [ ]:
biosample_contexts_frame['predicted_env_package'] = package_predictions

**filter and count, then use `biosample_contexts_frame` as an approximation of `local/nmdc-production-biosamples-soil-env_broad_scale.tsv`**

In [ ]:
# Filter and count values, then convert to DataFrame
soil_counts_df = biosample_contexts_frame[biosample_contexts_frame['predicted_env_package'] == 'soil'][
    'env_broad_scale_id'].value_counts().reset_index()

# Rename columns for clarity
soil_counts_df.columns = ['curie', 'count']


In [ ]:
soil_counts_df

In [ ]:
ncbi_frame = ncbi_conn.execute(ncbi_query).fetchdf()

In [ ]:
ncbi_frame.insert(0, 'serial_number', range(1, len(ncbi_frame) + 1))

In [ ]:
# includes env broad scale values with counts of one... useful for discovering drag-down submissions?

In [ ]:
ncbi_frame['content_list'] = ncbi_frame['content'].str.split('|')

In [ ]:
ncbi_frame['content_count'] = ncbi_frame['content_list'].apply(len)

In [ ]:
ncbi_frame.shape

In [ ]:
ncbi_frame = ncbi_frame.explode('content_list').reset_index(drop=True)

In [ ]:
ncbi_frame.shape

In [ ]:
# how many content_list strings contain envo multiple times now?

In [ ]:
ncbi_frame['envo_count'] = ncbi_frame['content_list'].str.lower().str.count("envo")

In [ ]:
ncbi_frame['envo_count'].value_counts()

doesn't account for multiple label strings delimited with something other than '|'

In [ ]:
ncbi_frame[['extracted_label', 'extracted_curie']] = ncbi_frame['content_list'].apply(parse_curie_label)

In [ ]:
parse_failures = ncbi_frame[(ncbi_frame['envo_count'] > 0) & (ncbi_frame['extracted_curie'].isna() | (ncbi_frame['extracted_curie'] == ''))]


In [ ]:
parse_failures

In [ ]:
ncbi_frame['real_label'] = ncbi_frame['extracted_curie'].apply(envo_adapter.label)

In [ ]:
# Apply the function to each row in the 'label' column
ncbi_frame['longest_annotation_curie'] = ncbi_frame['extracted_label'].apply(lambda x: get_longest_annotation_curie(x, envo_adapter))


In [ ]:
ncbi_frame['longest_annotation_label'] = ncbi_frame['longest_annotation_curie'].apply(envo_adapter.label)

In [ ]:
ncbi_frame

**Use `ncbi_frame` as an approximation of `local/ncbi-mims-soil-biosamples-env_broad_scale-annotated.tsv`**

In [ ]:
gold_biosamples_frame = pd.read_excel(gold_data_url, sheet_name=BIOSAMPLES_SHEET)
# 2 minutes

In [ ]:
gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'] = gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'].fillna(
    0).astype(int)


In [ ]:
# gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'] = gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'].astype(int)

In [ ]:
gold_biosamples_frame.columns

In [ ]:
gold_biosamples_frame.shape

In [ ]:
gold_biosamples_frame

In [ ]:
# Determine the filenames and target directory
goldterms_compressed_filename = urlparse(goldterms_semsql_url).path.split('/')[-1]
goldterms_filename = os.path.splitext(goldterms_compressed_filename)[0]
target_dir = os.path.join("..", "..")  # Two levels up

# Print to confirm the filenames
print(goldterms_filename)

In [ ]:
# Fetch the contents from the URL and save compressed file in target directory
goldterms_response = requests.get(goldterms_semsql_url)
goldterms_compressed_file_path = os.path.join(target_dir, goldterms_compressed_filename)
with open(goldterms_compressed_file_path, "wb") as f:
    f.write(goldterms_response.content)

In [ ]:
# Unzip the compressed file and save the extracted file in target directory
goldterms_uncompressed_file_path = os.path.join(target_dir, goldterms_filename)
with gzip.open(goldterms_compressed_file_path, "rb") as f_in:
    with open(goldterms_uncompressed_file_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
goldterms_conn = sqlite3.connect(goldterms_uncompressed_file_path)

In [ ]:
goldterms_soil_subjects = pd.read_sql_query(goldterms_soil_subclass_query, goldterms_conn)

In [ ]:
goldterms_soil_subjects['path_id'] = goldterms_soil_subjects['subject'].str.extract(r'GOLDTERMS:(\d+)')

In [ ]:
goldterms_soil_subjects

In [ ]:
soil_path_ids = goldterms_soil_subjects['path_id'].dropna().unique().tolist()
soil_path_ids = [int(id) for id in soil_path_ids]


In [ ]:
gold_soil_biosamples_frame = gold_biosamples_frame[
    gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'].isin(soil_path_ids)]


In [ ]:
gold_soil_biosamples_frame

In [ ]:
goldterms_mixs_broad_frame = pd.read_sql_query(goldterms_envo_query, goldterms_conn)

In [ ]:
goldterms_mixs_broad_frame['mixs_broad_label'] = goldterms_mixs_broad_frame['object'].apply(envo_adapter.label)

In [ ]:
goldterms_mixs_broad_frame['path_id'] = goldterms_mixs_broad_frame['subject'].str.extract(r'GOLDTERMS:(\d+)')

In [ ]:
goldterms_mixs_broad_frame

In [ ]:
# Fill NaN values in 'BIOSAMPLE ECOSYSTEM PATH ID' with 0 and convert to int
gold_soil_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'] = gold_soil_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'].fillna(0).astype(int)

# Drop rows with NaN in 'path_id' in goldterms_mixs_broad_frame
goldterms_mixs_broad_frame = goldterms_mixs_broad_frame.dropna(subset=['path_id'])

# Convert 'path_id' to int
goldterms_mixs_broad_frame['path_id'] = goldterms_mixs_broad_frame['path_id'].astype(int)

# Perform the left merge
gold_soil_biosamples_inferred_broad = gold_soil_biosamples_frame.merge(
    goldterms_mixs_broad_frame,
    left_on='BIOSAMPLE ECOSYSTEM PATH ID',
    right_on='path_id',
    how='left'
)


In [ ]:
gold_soil_biosamples_inferred_broad

**Use `gold_soil_biosamples_inferred_broad` as an approximation of `local/goldData_biosamples-inferred-soil-env_broad_scale-counts.tsv`**